In [6]:
import math

In [17]:
import src, banana.foo

In [18]:
print(dir(banana))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'foo']
